In [1]:
# %load_ext tensorboard


# import tensorflow as tf
# import tensorflow_addons as tfa
# from tensorflow import keras
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# from sklearn import preprocessing
# from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
# from sklearn import metrics
# from sklearn.metrics import classification_report


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D
from keras import backend as K
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import os
import psutil
import csv
from itertools import repeat
from PIL import Image
from numpy import asarray
from scipy.stats import entropy
import math

In [3]:
try:
  df = pd.read_csv("file5.csv",index_col=False, skipinitialspace=True)
except:
  df = pd.read_csv("C:\\Users\\jpath\\Project\\heart_cleaned.csv", index_col=False, skipinitialspace=True)

In [4]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,28.32,No,No,No,0,10,No,Female,70-74,White,No,No,Very good,9,No,No,No
1,No,26.13,No,No,No,0,0,No,Female,65-69,White,No,Yes,Very good,6,No,No,No
2,No,36.02,No,No,No,0,0,No,Female,70-74,White,Yes,Yes,Very good,7,Yes,No,No
3,No,33.64,No,No,No,29,3,Yes,Female,80 or older,White,Yes,No,Fair,8,No,No,No
4,No,21.59,No,No,No,0,3,No,Female,45-49,White,No,Yes,Excellent,6,No,No,No


In [5]:
df = df.replace('?', np.nan)
df[pd.isnull(df).any(axis=1)].shape

(0, 18)

In [6]:
print(df.shape)

#'number_outpatient', ,'number_inpatient'

(89997, 18)


In [7]:
df['AgeCategory'].value_counts()  

65-69          9647
60-64          9378
70-74          8716
55-59          8569
80 or older    7089
50-54          7051
45-49          6077
18-24          6016
75-79          5859
40-44          5815
35-39          5766
30-34          5257
25-29          4757
Name: AgeCategory, dtype: int64

In [8]:
# categorizing age
#age <50,>50
df.loc[df['AgeCategory']== '18-24','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '25-29','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '30-34','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '35-39','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '40-44','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '45-49','AgeCategory'] = 0;
df.loc[df['AgeCategory']== '50-54','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '55-59','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '60-64','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '65-69','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '70-74','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '75-79','AgeCategory'] = 1;
df.loc[df['AgeCategory']== '80 or older','AgeCategory'] = 1;

df['AgeCategory'] = df['AgeCategory'].astype(np.int32)

In [9]:
df['AgeCategory'].value_counts()

1    56309
0    33688
Name: AgeCategory, dtype: int64

In [10]:
df['Race'].value_counts()

White                             70734
Hispanic                           7014
Black                              6376
Other                              2391
American Indian/Alaskan Native     1951
Asian                              1531
Name: Race, dtype: int64

In [11]:
#converting race to binary

df.loc[df['Race']== 'White','Race'] = 'White';
df.loc[df['Race']== 'Black','Race'] = 'Non-White';
df.loc[df['Race']== 'Hispanic','Race'] = 'Non-White';
df.loc[df['Race']== 'Other','Race'] = 'Non-White';
df.loc[df['Race']== 'Asian','Race'] = 'Non-White';
df.loc[df['Race']== 'American Indian/Alaskan Native','Race'] = 'Non-White';

In [12]:
df['Race'].value_counts()

White        70734
Non-White    19263
Name: Race, dtype: int64

In [13]:
df['Sex'].value_counts()

Female    47121
Male      42876
Name: Sex, dtype: int64

In [14]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,28.32,No,No,No,0,10,No,Female,1,White,No,No,Very good,9,No,No,No
1,No,26.13,No,No,No,0,0,No,Female,1,White,No,Yes,Very good,6,No,No,No
2,No,36.02,No,No,No,0,0,No,Female,1,White,Yes,Yes,Very good,7,Yes,No,No
3,No,33.64,No,No,No,29,3,Yes,Female,1,White,Yes,No,Fair,8,No,No,No
4,No,21.59,No,No,No,0,3,No,Female,0,White,No,Yes,Excellent,6,No,No,No


In [15]:
#Normalization
normalize_columns = ['BMI','PhysicalHealth', 'MentalHealth', 'SleepTime']

In [16]:
def show_values(columns):
  for column in columns:
    max_val = df[column].max()
    min_val = df[column].min()
    mean_val = df[column].mean()
    var_val = df[column].var()
    print(column +': values=['+str(min_val)+','+str(max_val)+'] , mean='+str(mean_val)+' , var='+str(var_val))

In [17]:
show_values(normalize_columns)

BMI: values=[12.02,92.53] , mean=28.394180917141682 , var=40.42965978428491
PhysicalHealth: values=[0,30] , mean=3.3234441148038267 , var=62.99415379869125
MentalHealth: values=[0,30] , mean=3.87357356356323 , var=63.39779012402347
SleepTime: values=[1,24] , mean=7.098225496405436 , var=1.997640857503177


In [18]:
def normalize(columns):
  scaler = preprocessing.StandardScaler()
  df[columns] = scaler.fit_transform(df[columns])
normalize(normalize_columns)
show_values(normalize_columns)

BMI: values=[-2.5752058724148794,10.086790830643045] , mean=6.126661588162247e-17 , var=1.0000111116050023
PhysicalHealth: values=[-0.418736356589204,3.3611047551418483] , mean=-1.705359411137945e-17 , var=1.0000111116047676
MentalHealth: values=[-0.486493633293903,3.281295659803087] , mean=1.705359411137945e-17 , var=1.0000111116047576
SleepTime: values=[-4.314666037671566,11.958480785931608] , mean=-2.652781306214581e-16 , var=1.0000111116061225


In [19]:
categorical_columns = ['Smoking', 'AlcoholDrinking','Stroke', 'DiffWalking', 'Sex', 'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease',  'SkinCancer']
label_column = ['HeartDisease']

# Smoking	AlcoholDrinking	Stroke	PhysicalHealth	MentalHealth	DiffWalking	Sex	AgeCategory	Race	Diabetic	PhysicalActivity	GenHealth	SleepTime	Asthma	KidneyDisease	SkinCancer

In [20]:
def show_unique_values(columns):
  for column in columns:
    uniq = df[column].unique().tolist()
    print(column+ " has "+ str(len(uniq)) +" values" + " : " + str(uniq))
show_unique_values(categorical_columns)
show_unique_values(label_column)

Smoking has 2 values : ['No', 'Yes']
AlcoholDrinking has 2 values : ['No', 'Yes']
Stroke has 2 values : ['No', 'Yes']
DiffWalking has 2 values : ['No', 'Yes']
Sex has 2 values : ['Female', 'Male']
AgeCategory has 2 values : [1, 0]
Race has 2 values : ['White', 'Non-White']
Diabetic has 4 values : ['No', 'Yes', 'No, borderline diabetes', 'Yes (during pregnancy)']
PhysicalActivity has 2 values : ['No', 'Yes']
GenHealth has 5 values : ['Very good', 'Fair', 'Excellent', 'Poor', 'Good']
Asthma has 2 values : ['No', 'Yes']
KidneyDisease has 2 values : ['No', 'Yes']
SkinCancer has 2 values : ['No', 'Yes']
HeartDisease has 2 values : ['No', 'Yes']


In [21]:
def convert_to_onehot(data,columns):
  dummies = pd.get_dummies(data[columns])
  data = data.drop(columns, axis=1)
  data = pd.concat([data, dummies], axis=1)
  return data

In [22]:
df = convert_to_onehot(df,categorical_columns)
df = convert_to_onehot(df,label_column)

In [23]:
df.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,AgeCategory,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,Stroke_No,...,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes,HeartDisease_No,HeartDisease_Yes
0,-0.011667,-0.418736,0.769436,1.345559,1,1,0,1,0,1,...,0,1,1,0,1,0,1,0,1,0
1,-0.356093,-0.418736,-0.486494,-0.777025,1,1,0,1,0,1,...,0,1,1,0,1,0,1,0,1,0
2,1.199330,-0.418736,-0.486494,-0.069497,1,1,0,1,0,1,...,0,1,0,1,1,0,1,0,1,0
3,0.825022,3.235110,-0.109715,0.638031,1,1,0,1,0,1,...,0,0,1,0,1,0,1,0,1,0
4,-1.070110,-0.418736,-0.109715,-0.777025,0,1,0,1,0,1,...,0,0,1,0,1,0,1,0,1,0


In [24]:
print(df.shape)

(89997, 36)


In [25]:
from sklearn.model_selection import train_test_split

s_names = ['HeartDisease_Yes', 'HeartDisease_No']
# s_names = ['HeartDisease']
x_data = df.drop(s_names, axis=1)
y_labels = df[s_names]

print(x_data.shape, y_labels.shape)
xServer, X_test, yServer, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=123)



df.shape

(89997, 34) (89997, 2)


(89997, 36)

In [26]:
 algoName='CNN'
x_shape, y_shape =x_data.shape[1], y_labels.shape[1]
mean_shape = (x_shape + y_shape) // 2
#outputClasses=len(set(y_labels))
outputClasses=y_labels.shape[1]
def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    recall=recall_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("f1Score : {}".format(f1Score))
    
    
    return accuracy, precision, recall, f1Score

verbose, epochs, batch_size = 0, 10, 64
activationFun='relu'
optimizerName='Adam'
def createDeepModel():
    model = Sequential()
    
    if(algoName=='CNN'):    
        model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun,input_shape = (x_shape, 1))) # changed x_data.shape[1] to x_shape
        model.add(Dropout(0.1))
        model.add(MaxPooling1D(pool_size=3))
        model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun))
        model.add(Dropout(0.05))
        model.add(MaxPooling1D(pool_size=3))
        model.add(Flatten())  # Uncommented this line
        model.add(Dense(x_shape, activation=activationFun))
        model.add(Dense(mean_shape, activation=activationFun))
        model.add(Dense(50, activation=activationFun))
        model.add(Dense(outputClasses, activation='sigmoid'))
        model.compile(loss='mean_squared_error', optimizer=optimizerName, metrics=['accuracy'])

    return model
def predictTestData(yPredict, y_test): ##
    #Converting predictions to label
    
    print("yPredict",len(yPredict))
    pred = list()
    for i in range(len(yPredict)):
        pred.append(np.argmax(yPredict[i])) ###i
    #Converting one hot encoded test label to label
    test = list()
    for i in range(len(y_test)):
        test.append(np.argmax(y_test[i]))
    return my_metrics(test, pred)

def sumOfWeights(weights):
    return sum(map(sum, weights))

def getWeights(model):
    allLayersWeights=deepModel.get_weights()
    return allLayersWeights


    
# Initially train central deep model
deepModel=createDeepModel()

In [27]:
def getWeights(deepModel):
    allLayersWeights = deepModel.get_weights()
    return allLayersWeights
for layer in deepModel.layers:
    weights = layer.get_weights() # list of numpy arrays
    if len(weights) > 0:
        print("Layer:", layer.name)
        print("Weights:", weights[0]) # weights array
        print("Biases:", weights[1]) # bias array

Layer: conv1d
Weights: [[[-0.15008864 -0.3035834  -0.13948661 -0.07529593  0.2678334 ]]

 [[ 0.13929099 -0.33115464  0.28551698  0.03589267 -0.34804678]]

 [[ 0.01263964  0.29637462 -0.5127732  -0.19969517 -0.27991337]]]
Biases: [0. 0. 0. 0. 0.]
Layer: conv1d_1
Weights: [[[ 0.3298455  -0.19491598  0.367455    0.27788407  0.40245783]
  [-0.1135627   0.23037201 -0.27498168 -0.02553314  0.22427052]
  [ 0.3142014  -0.38125202  0.41843706 -0.13819209  0.24118823]
  [ 0.40774328 -0.390882    0.14485842  0.02986443  0.28092998]
  [-0.44238618 -0.3110804   0.12458909 -0.10416132 -0.4264878 ]]

 [[-0.16928568  0.18480068 -0.11452648 -0.31164634 -0.27068484]
  [ 0.31010586 -0.12261316 -0.29780388 -0.04975855 -0.1902659 ]
  [ 0.3940903   0.2594754   0.2326563  -0.06551075  0.20872611]
  [-0.06343564  0.3913343   0.16537225  0.18940908 -0.31265843]
  [ 0.30512792  0.4382096   0.03618276 -0.23658358 -0.12400013]]

 [[ 0.10313231  0.2450164   0.24345368 -0.11561042 -0.04708657]
  [ 0.14872807  0.348

In [28]:
getWeights(deepModel)

[array([[[-0.15008864, -0.3035834 , -0.13948661, -0.07529593,
           0.2678334 ]],
 
        [[ 0.13929099, -0.33115464,  0.28551698,  0.03589267,
          -0.34804678]],
 
        [[ 0.01263964,  0.29637462, -0.5127732 , -0.19969517,
          -0.27991337]]], dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32),
 array([[[ 0.3298455 , -0.19491598,  0.367455  ,  0.27788407,
           0.40245783],
         [-0.1135627 ,  0.23037201, -0.27498168, -0.02553314,
           0.22427052],
         [ 0.3142014 , -0.38125202,  0.41843706, -0.13819209,
           0.24118823],
         [ 0.40774328, -0.390882  ,  0.14485842,  0.02986443,
           0.28092998],
         [-0.44238618, -0.3110804 ,  0.12458909, -0.10416132,
          -0.4264878 ]],
 
        [[-0.16928568,  0.18480068, -0.11452648, -0.31164634,
          -0.27068484],
         [ 0.31010586, -0.12261316, -0.29780388, -0.04975855,
          -0.1902659 ],
         [ 0.3940903 ,  0.2594754 ,  0.2326563 , -0.06551075,
      

In [29]:
deepModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 32, 5)             20        
                                                                 
 dropout (Dropout)           (None, 32, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 8, 5)              80        
                                                                 
 dropout_1 (Dropout)         (None, 8, 5)              0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2, 5)             0         
 1D)                                                    

In [30]:
numOfIterations= 10
numOfClients= 5  # 15, 20, 25, 30, 35, 40, 45, 50
modelLocation='C:\\Users\\PRAKHAR\\Documents\\GitHub\\dsci2023\\saved_model\\saved_model'+str(algoName)+"_Sync_users_"+str(numOfClients)+"_"+activationFun+"_"+optimizerName+"_FL_HeartModel.h5"
accList, precList, recallList, f1List = [], [], [], []

deepModelAggWeights=[]
firstClientFlag=True

def updateServerModel(clientModel, clientModelWeight):
    global firstClientFlag
    for ind in range(len(clientModelWeight)):
        if(firstClientFlag==True):
            deepModelAggWeights.append(clientModelWeight[ind])            
        else:
            deepModelAggWeights[ind]=(deepModelAggWeights[ind]+clientModelWeight[ind])

def updateClientsModels():
    global clientsModelList
    global deepModel
    clientsModelList.clear()
    for clientID in range(numOfClients):
        m = keras.models.clone_model(deepModel)
        m.set_weights(deepModel.get_weights())
        clientsModelList.append(m)

In [31]:
# ----- 1. Train central model initially -----
def trainInServer():
    deepModel.fit(xServer, yServer, epochs=epochs, batch_size=batch_size)
    #deepModel.fit(X_full, Y_full, epochs=epochs, batch_size=batch_size, verbose=verbose)
    deepModel.save("final_model.h5")
trainInServer()



Epoch 1/10
985/985 [==============================] - 3s 2ms/step - loss: 0.0825 - accuracy: 0.9138
Epoch 2/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0751 - accuracy: 0.9138
Epoch 3/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0736 - accuracy: 0.9138
Epoch 4/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0729 - accuracy: 0.9138
Epoch 5/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0724 - accuracy: 0.9138
Epoch 6/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0718 - accuracy: 0.9138
Epoch 7/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0717 - accuracy: 0.9138
Epoch 8/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0716 - accuracy: 0.9138
Epoch 9/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0717 - accuracy: 0.9138
Epoch 10/10
985/985 [==============================] - 2s 2ms/step - loss: 0.0717 - accuracy: 0.9138